In [5]:
import pandas as pd
import re


movies = pd.read_json('data/IMDB_movie_details_eng.json')
reviews = pd.read_json('data/IMDB_reviews.json',lines = True)

movies.movie_id = movies.movie_id.astype('str')
reviews.movie_id = reviews.movie_id.astype('str')

df = pd.merge(movies, reviews, on='movie_id')


In [6]:
df_filtered = df[df.is_spoiler ==True]
# take one movie per genre
df_filtered = df_filtered.drop_duplicates(subset='genre', keep='first').reset_index(drop=True)
df_filtered.head()

,movie_id,plot_summary,duration,genre,rating_x,release_date,plot_synopsis,title,review_date,user_id,is_spoiler,review_text,rating_y,review_summary
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in...",Patriot Games,5 March 2008,ur16517420,True,The second Tom Clancy novel made into a film (...,6,decent Clancy thriller
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...,Last Vegas,2 June 2014,ur5291991,True,Last Vegas is a comedy that features an ensemb...,8,"Douglas,De Niro,Freeman and Kline Are Simply H..."
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,,Wet Hot American Summer,28 July 2015,ur3728510,True,"""Not ANOTHER 'Another Teen Movie,'"" is the acc...",8,Smells like teen spirit or is that the refrie...
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...,The Treasure of the Sierra Madre,27 June 2004,ur1406078,True,John Huston's genius as a director is undeniab...,10,"All that glitters is ""gone with the wind...."""
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...,Election,16 January 2009,ur8239592,True,"Popular, but frustrated high school civics tea...",9,A splendidly wicked satirical treat


In [7]:
df_filtered.drop(['duration', 'review_summary',  'rating_y', 'rating_x'], axis=1, inplace=True)

In [8]:
# fill with null when plot_synopsis is empty
df_filtered['plot_synopsis'] = df_filtered['plot_synopsis'].apply(lambda x: x if x != '' else None)

In [9]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   movie_id       237 non-null    object
 1   plot_summary   237 non-null    object
 2   genre          237 non-null    object
 3   release_date   237 non-null    object
 4   plot_synopsis  198 non-null    object
 5   title          237 non-null    object
 6   review_date    237 non-null    object
 7   user_id        237 non-null    object
 8   is_spoiler     237 non-null    bool  
 9   review_text    237 non-null    object
dtypes: bool(1), object(9)
memory usage: 17.0+ KB


In [10]:
df_filtered

,movie_id,plot_summary,genre,release_date,plot_synopsis,title,review_date,user_id,is_spoiler,review_text
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...","[Action, Thriller]",1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in...",Patriot Games,5 March 2008,ur16517420,True,The second Tom Clancy novel made into a film (...
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",[Comedy],2013-11-01,Four boys around the age of 10 are friends in ...,Last Vegas,2 June 2014,ur5291991,True,Last Vegas is a comedy that features an ensemb...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...","[Comedy, Romance]",2002-04-11,None,Wet Hot American Summer,28 July 2015,ur3728510,True,"""Not ANOTHER 'Another Teen Movie,'"" is the acc..."
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...","[Adventure, Drama, Western]",1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...,The Treasure of the Sierra Madre,27 June 2004,ur1406078,True,John Huston's genius as a director is undeniab...
4,tt0126886,Tracy Flick is running unopposed for this year...,"[Comedy, Drama, Romance]",1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...,Election,16 January 2009,ur8239592,True,"Popular, but frustrated high school civics tea..."
...,...,...,...,...,...,...,...,...,...,...
232,tt0259711,"Incarcerated and charged with murder, David Aa...","[Fantasy, Mystery, Romance]",2001-12-14,"David Aames (Tom Cruise) drives to work, he fi...",Vanilla Sky,4 December 2003,ur2688719,True,I for one really like this movie for some reas...
233,tt0271263,Adam Sandler invites you to share some holiday...,"[Animation, Comedy, Musical]",2002-11-27,None,Eight Crazy Nights,11 November 2003,ur0697173,True,I had no choice but to watch this movie at a f...
234,tt0993846,Jordan Belfort is a Long Island penny stockbro...,"[Biography, Comedy, Crime]",2013-12-25,The movie opens with a TV advertisement for St...,The Wolf of Wall Street,24 June 2017,ur65913028,True,"Let me start by saying that in my view, Scorse..."
235,tt1707386,"Jean Valjean, known as Prisoner 24601, is rele...","[Drama, History, Musical]",2012-12-25,[This version is a close adaptation of the Bro...,Les Misérables,22 December 2012,ur2979676,True,The guiding ethic of any film adaptation of a ...


### Chat

In [4]:
%%capture
!pip install openai

In [ ]:
from openai import OpenAI
import time

api_key = 'api-key-here'
client = OpenAI(api_key = api_key)


count = 0
def call_api(summary, synopsis, review, model = 'gpt-3.5-turbo'):
    global count
    count += 1
    error = None
    if count % 10 == 0:
        print(f"Processing {count}th review")
    try:
        response = client.chat.completions.create(model=model,
        messages=[{"role": "assistant", "content": f" You are an assistant specializing in spoiler detection for reviews. A spoiler is any information about the plot of a TV series, film, or book that could ruin the enjoyment for someone who has not yet seen or read it. You have this task: 1) Rephrase the review to remove any spoilers while maintaining the original wording. To identify spoilers, consider any plot elements mentioned in the plot synopsis but not in the summary. Plot summary: {summary}, Plot synopsis: {synopsis}"},
              {"role": "user", "content": f" {review}"}],
        max_tokens = 1024,
        temperature = 0.001,
        top_p = 0.9)

        # first output: paraphrased review
        first_output = response.choices[0].message.content
        # take only what comes after \n\n
        #first_output = first_output.split('\\n\\n')[1:]
        #first_output = ' '.join(first_output)
        # remove the \ using regex
        #first_output = re.sub(r'\\', '', first_output)

    except (Exception or UnboundLocalError) as e:
        print(f"Error: {e} at {count}th review for first api call")
        first_output = None
        error = e


    try:
        response = client.chat.completions.create(model="gpt-3.5-turbo", messages = [
            {"role": "assistant",
                "content": f"You are an assistant specializing in spoiler detection for reviews. You are given a plot summary without spoilers to understand the concepts, a plot synopsis with spoilers to understand how to remove spoilers. Plot summary: {summary}, Plot synopsis: {synopsis}."},
            {"role": "user",
                "content": f"Return the phrases that are more spoiler from this review: {review}. Return the results in a numbered format."
            }],
        max_tokens = 500,
        temperature = 0.0,
        top_p = 1.0)
        second_output = response.choices[0].message.content
        second_output = re.sub(r'\d+\.\s', '', second_output)
        second_output_list = second_output.split('\n')
        for i in second_output_list:
            masked_output = review.replace(i, '#SPOILER#')
            non_masked_output = review.replace(i, '')

    except (Exception or UnboundLocalError) as e:
        print(f"Error: {e} at {count}th review for second api call")
        masked_output = None
        non_masked_output = None
        error = e

    output = (first_output, masked_output, non_masked_output)

    return output, error

# Iterate over each row in the DataFrame and call the API
results_rephrase = []
results_masked = []
results_non_masked = []

t = 60
for i in range(1,5):
    df_slice = df_filtered.iloc[(i-1)*50:i*50]
    for _, row in df_slice.iterrows():
        # add 10 seconds sleep to avoid rate limit
        time.sleep(t)
        summary = row['plot_summary']
        synopsis = row['plot_synopsis']
        review = row['review_text']

        response_content, error = call_api(summary, synopsis, review)
        if error is not None:
            t = 0
            break
        results_rephrase.append(response_content[0])
        results_masked.append(response_content[1])
        results_non_masked.append(response_content[2])
    if i == 4:
        df_slice = df_filtered.iloc[200:]
        for _, row in df_slice.iterrows():
            # add 10 seconds sleep to avoid rate limit
            time.sleep(t)
            summary = row['plot_summary']
            synopsis = row['plot_synopsis']
            review = row['review_text']

            response_content, error = call_api(summary, synopsis, review)
            if error is not None:
                t = 0
                break
            results_rephrase.append(response_content[0])
            results_masked.append(response_content[1])
            results_non_masked.append(response_content[2])

pad_dim = len(df_filtered) - len(results_rephrase)


In [12]:
print(pad_dim)
df_filtered['rephrased_review'] = results_rephrase + [None] * pad_dim
df_filtered['masked_review'] = results_masked + [None] * pad_dim
df_filtered['non_masked_review'] = results_non_masked + [None] * pad_dim

144


In [13]:
df_filtered.head()

,movie_id,plot_summary,genre,release_date,plot_synopsis,title,review_date,user_id,is_spoiler,review_text,rephrased_review,masked_review,non_masked_review
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...","['Action', 'Thriller']",1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in...",Giochi di potere,5 March 2008,ur16517420,True,The second Tom Clancy novel made into a film (...,In the second film adaptation of a Tom Clancy ...,The second Tom Clancy novel made into a film (...,The second Tom Clancy novel made into a film (...
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",['Comedy'],2013-11-01,Four boys around the age of 10 are friends in ...,Last Vegas,2 June 2014,ur5291991,True,Last Vegas is a comedy that features an ensemb...,"""Last Vegas"" is a comedy featuring a star-stud...",Last Vegas is a comedy that features an ensemb...,Last Vegas is a comedy that features an ensemb...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...","['Comedy', 'Romance']",2002-04-11,NaN,Wet Hot American Summer,28 July 2015,ur3728510,True,"""Not ANOTHER 'Another Teen Movie,'"" is the acc...","""Wet Hot American Summer"" is a film that defie...","""Not ANOTHER 'Another Teen Movie,'"" is the acc...","""Not ANOTHER 'Another Teen Movie,'"" is the acc..."
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...","['Adventure', 'Drama', 'Western']",1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...,Il tesoro della Sierra Madre,27 June 2004,ur1406078,True,John Huston's genius as a director is undeniab...,John Huston's talent as a director shines thro...,John Huston's genius as a director is undeniab...,John Huston's genius as a director is undeniab...
4,tt0126886,Tracy Flick is running unopposed for this year...,"['Comedy', 'Drama', 'Romance']",1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...,Election,16 January 2009,ur8239592,True,"Popular, but frustrated high school civics tea...","In this sharp and witty high school comedy, a ...","Popular, but frustrated high school civics tea...","Popular, but frustrated high school civics tea..."


In [15]:
df_filtered.to_csv('openai_reviews.csv', index=False)